In [16]:
from sklearn.datasets import load_wine
x, y = load_wine(return_X_y = True)

In [17]:
x = x[:59+71]
y = y[:59+71]

In [18]:
from sklearn.model_selection import train_test_split
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9, shuffle = True, stratify = y)

In [19]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [20]:
import numpy as np
x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)

In [21]:
import pennylane as qml
ncubits = 4
dev = qml.device("lightning.qubit", wires = ncubits)
@qml.qnode(dev)
def circuito_kernel(a, b):
    qml.AmplitudeEmbedding(
        b, wires = range(ncubits), pad_with = 0, normalize = True
    )
    qml.adjoint(qml.AmplitudeEmbedding(
        a, wires = range(ncubits), pad_with = 0, normalize = True 
    ))
    return qml.probs(wires = range(ncubits))
    

In [22]:
def qkernel(A, B): # A, B son arrays de vectores
    return np.array([[circuito_kernel(a, b)[0] for b in B] for a in A])
    # Devuelve una matriz X donde X[i,j] = A[i] * B[j]

In [23]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [24]:
svm = SVC(kernel = qkernel).fit(x_tr, y_tr)
print(accuracy_score(svm.predict(x_test), y_test))

0.9230769230769231


In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 8)
xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

In [11]:
ncubits = 8
dev = qml.device("lightning.qubit", wires = ncubits)

@qml.qnode(dev)
def circuito_kernel(a, b):
    qml.AngleEmbedding(b, wires = range(ncubits))
    qml.adjoint(qml.AngleEmbedding(a, wires = range(ncubits)))
    return qml.probs(wires = range(ncubits))


In [12]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))
print(accuracy_score(svm.predict(xs_tr), y_tr))

1.0
0.9743589743589743


In [13]:
from itertools import combinations
def CodificadorZZ(ncubits, datos):
    ncarga = min(len(datos), ncubits)
    for i in range(ncarga):
        qml.Hadamard(wires = i)
        qml.RZ(2.0 * datos[i], wires = i)
    for par in list(combinations(range(ncubits), 2)):
        q0 = par[0]
        q1 = par[1]
        qml.CNOT(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - datos[q0]) * (np.pi - datos[q1]), wires = q1)
        qml.CNOT(wires = [q0, q1])

In [14]:
ncubits = 8
dev = qml.device("lightning.qubit", wires = ncubits)

@qml.qnode(dev)
def circuito_kernel(a, b):
    CodificadorZZ(ncubits, b)
    qml.adjoint(CodificadorZZ)(ncubits, a)
    return qml.probs(wires = range(ncubits))

In [15]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))
print(accuracy_score(svm.predict(xs_tr), y_tr))

1.0
1.0
